In [162]:
import re
import os
import json
import time

def get_text(text):
    text = re.sub('\n+', ' ', text.strip())
    text = re.sub('\r+', ' ', text.strip())
    text = ' '.join(map(lambda x: x.strip(), text.strip().split('\r\n')))
    clean_text, start = '', 0
    while text:
        text = text.split('<', 1)
        clean_text += text[0]
        if len(text) < 2:
            break
        tmp = text[1].split('>', 1)
        if len(tmp) < 2:
            break
        text = tmp[1]
    clean_text = re.sub(' +', ' ', clean_text)
    clean_text = unicode(clean_text, errors='ignore')
    return clean_text


def parse_item(filename):
    filetype = filename.split('/')[-1]
    date = ''.join(filename.split('/evolution/')[-1].split('/')[0].split('-'))
    
    if filetype == 'feedback' or filetype == 'return-policy':
        itemID = date + filename.split('/')[-2]
    else:
        itemID = date + filename.split('/')[-1]

    # filename='/media/intel/m2/evolution/2014-06-04/listing/12030'
    with open(filename) as fp:
        content = fp.read()

    if (not content) or content.find('Error in exception handler') >= 0:
        return None
    
    title, content = content.split('<title>')[1].split('</title>')
    title = get_text(title)
    if title == 'Evolution :: Login':
        return None
    
    tmp = content.split('>Home</a></li>    <li><a')
    if len(tmp) == 1:
        return None
    cat, content = tmp[1].split('>', 1)[1].split('<', 1)
    
    tmp = content.split('<a class="thumbnail" href="', 1)
    if len(tmp) == 1:
        return None
    tmp = tmp[1].split('"', 1)
    if len(tmp) == 1:
        return None
    img_path, content = tmp
    img_path = None if 'none' in img_path.split('/') else img_path.split('/evolution/')[-1]
    if img_path is not None:
        img_path = img_path.split('.onion/')[-1]

    if date > '20140809':
        tmp = content.split('<p>By <a')
        if len(tmp) == 1:
            return None
        seller, content = tmp[1].split('>', 1)[1].split('</a>', 1)
    else:
        tmp = content.split('<div class="seller-info text-muted">Seller:')
        if len(tmp) == 1:
            return None
        seller, content = tmp[1].split('</a>', 1)
    seller = get_text(seller)
    
    tmp = content.split('>Level', 1)
    if len(tmp) == 1:
        item_sold = None
    else:
        item_sold, content = tmp[1].split('(', 1)[1].split(')', 1)
        item_sold = int(item_sold)
    
    tmp = content.split('>BTC ', 1)
    if len(tmp) == 1:
        price_btc = None
    else:
        price_btc, content = tmp[1].split('<', 1)
        price_btc = float(price_btc)
    
    if date > '20140809':
        tmp = content.split('>Ships From<')
        if len(tmp) == 1:
            ship_from = None
        else:
            ship_from, content = tmp[1].split('<', 1)[1].split('<', 1)
            # print ship_from
            ship_from = get_text('<' + ship_from)
    
    if len(content.split('>Description<', 1)) > 1:
        tmp = content.split('>Description<', 1)[1].split('>', 1)[1].split('</div>', 1)
        if len(tmp) == 1:
            raw_text = tmp[0]
        else:
            raw_text, content = tmp
        # return raw_text
        tmp = raw_text.split('>Ships To<')
        if len(tmp) == 1:
            text = get_text(tmp[0])
        else:
            text = get_text(tmp[0] + '>')
        text = text.strip()
    else:
        text = None
    
    if date <= '20140809':
        tmp = content.split('>Ships From<')
        if len(tmp) == 1:
            ship_from = None
        else:
            ship_from, content = tmp[1].split('<', 1)[1].split('<', 1)
            # print ship_from
            ship_from = get_text('<' + ship_from)
        
    iteminfo = {}
    iteminfo['title'] = title
    iteminfo['itemID'] = itemID
    iteminfo['btc_price'] = price_btc
    iteminfo['cat'] = cat
    iteminfo['description'] = text
    iteminfo['seller'] = seller
    iteminfo['seller_dealmade'] = item_sold
    iteminfo['seller_location'] = ship_from
    iteminfo['date'] = date
    iteminfo['img'] = img_path
    
    return iteminfo

In [163]:
root_path = '/media/intel/m2/evolution/'
item = {}
error_count = 0

In [164]:
def analyze(path):
    # print path
    global error_count
    webpage_folder_path = os.path.join(path, 'listing')
    if not os.path.isdir(webpage_folder_path):
        return None
    filenames = map(lambda wp: os.path.join(path, 'listing', wp),
                    os.listdir(webpage_folder_path))
    for filename in filenames:
        try:
            if not os.path.isfile(filename):
                filename = os.path.join(filename, 'feedback')
                if not os.path.isfile(filename):
                    filename = filename.replace('feedback', 'return-policy')
            parse_res = parse_item(filename)
            if parse_res is not None:
                item[filename] = parse_res               
        except:
            print filename
            error_count += 1
            if error_count == 10000:
                break
    return 1

In [165]:
paths = [os.path.join(root_path, x) for x in os.listdir(root_path)]
paths = sorted(paths)

In [166]:
time0 = time.time()
for path in paths:
    if os.path.isdir(path):
        analyze(path)
        if error_count == 10000:
            break
print time.time() - time0

499.543804884


In [167]:
len(item)

904037

In [168]:
with open('evolution.json', 'w') as fp:
    json.dump(item, fp)

In [117]:
parse_item('/media/intel/m2/evolution/2014-10-04/listing/41764')

{'btc_price': 0.7661,
 'cat': 'Drugs',
 'date': '20141004',
 'description': 'PROMOTION PRICE only until 01.11.14! Very strong and absolutley clean! This is the *Alice in wonderland* print. This print is exactly same strong as the loved *Octopus* print. These blotters got two drops of LSD, because of that they are very strong. If you are not sure take only 1/4 or 1/2.',
 'img': 'http://k5zq47j6wd3wdvjq.onion/static/image/111069/gymEnLZQbFHkejk9lD2YTnM396cZnaPB/0/large.jpg',
 'itemID': '2014100441764',
 'seller': 'McDrugs',
 'seller_dealmade': 24,
 'seller_location': 'Germany',
 'title': 'Evolution :: 50x 300&micro;g - LSD Blotter *Alice in wonderland* DoubleDip'}

In [118]:
parse_item('/media/intel/m2/evolution/2014-07-28/listing/9767')

{'btc_price': 0.8283,
 'cat': 'Drugs',
 'date': '20140728',
 'description': '25 grams of pure white PB-22 with shipping from US and a better price than the competition! :-) Fast super-stealth-shipping of a high quality pure material! ALL compounds are measured OVERWEIGHT. :-) (Please message if you desire a larger or smaller weight, from 5 to 250 grams) We&#039;re back! We were a long standing vendor on BMR with an excellent reputation. Check out our feedback here: http://loow5bjd6qjbwi5z.onion/index.php?p=user&amp;id=313230 We still use the same public PGP key so you needn&#039;t worry that we&#039;re not the real oxymorpheus. It is our intent to provide the best quality, the best service and the best price. We will beat any other US domestic vendors prices on ANY of our items here at evolution. We are offering Priority shipping for this bulk amounts because of the greater need to have tracking for an expensive shipment. Our stealth is the BEST. Nothing is more important',
 'img': 'ht

In [119]:
parse_item('/media/intel/m2/evolution/2014-07-26/listing/22354/feedback')

{'btc_price': 0.0581,
 'cat': 'Counterfeits',
 'date': '20140726',
 'description': None,
 'img': 'http://k5zq47j6wd3wdvjq.onion/static/image/42127/fd8s0PgFGZGnBsIymx83YlhLV3nhGhOH/0/large.jpg',
 'itemID': '2014072622354',
 'seller': 'MagicHat',
 'seller_dealmade': 59,
 'seller_location': None,
 'title': 'Evolution :: Listing :: RayBan Aviator Sunglasses'}

In [120]:
parse_item('/media/intel/m2/evolution/2014-06-24/listing/7678')

{'btc_price': 0.4313,
 'cat': 'Drugs',
 'date': '20140624',
 'description': '100 MG&#039;s for a good price. Due to the small size we are able to ship 100 in one regular envelope! Before ordering please read our profile and make sure you take the following rules into account: - By ordering you agree to all our terms written in our profile. - Format your address correct or we cancel your order. - No shipping to USA, Australia, New Zealand and Scandinavia under any circumstances! - We don&#039;t use escrow under any circumstances, sorry.',
 'img': 'http://k5zq47j6wd3wdvjq.onion/static/image/17538/m1VPnswgrecboz2ZwQBJSCY3BpJFpw6q/0/large.jpg',
 'itemID': '201406247678',
 'seller': 'AlbertHeijn',
 'seller_dealmade': 575,
 'seller_location': 'Netherlands',
 'title': 'Evolution :: Listing :: 100 x - Herbalife&#039;s (100MG) - Free Shipping'}

In [115]:
parse_item('/media/intel/m2/evolution/2014-01-21/listing/69')

In [113]:
parse_item('/media/intel/m2/evolution/2015-01-10/listing/71712/feedback')

{'btc_price': 0.0256,
 'cat': 'Weapons',
 'date': '20150110',
 'description': None,
 'img': 'http://k5zq47j6wd3wdvjq.onion/static/image/17841/IbWvrOBYkETvJ5CfKKursx9TAMDhlGRx/0/large.jpg',
 'itemID': '2015011071712',
 'seller': 'OzDomestic',
 'seller_dealmade': 134,
 'seller_location': 'Australia',
 'title': 'Evolution :: Pepper Spray Lighter - FE &amp; Free Post'}

In [114]:
parse_item('/media/intel/m2/evolution/2014-01-21/listing/15')

{'btc_price': 0.0168,
 'cat': 'Fraud Related',
 'date': '20140121',
 'description': 'A mix selection of CC with NON AVS BINS prices range from $14 depending on lookups !!! For fast reply times contact me on ICQ: 655581841 Thanks',
 'img': 'http://k5zq47j6wd3wdvjq.onion/image/77/EdaZpl2LH9jDd1RIjXA21ttrDXyy1PRd/0/large',
 'itemID': '2014012115',
 'seller': 'SuperRyanSmith33',
 'seller_dealmade': None,
 'seller_location': 'Worldwide',
 'title': 'Evolution :: Listing :: WORLD MIX NON AVS CC (FRESH 90%)'}

In [1]:
import pandas as pd
import re
import os
import json
import time
from shutil import copy2
import shutil

In [2]:
data = pd.read_json('evolution.json', orient='index')

len(data)

904037

In [3]:
len(data.seller.unique())

4197

In [3]:
df_unique = data.drop_duplicates(['seller', 'title'])

df_unique = df_unique[df_unique.img.notnull()]
df_unique = df_unique.sort_index()
df_unique['itemID'] = df_unique.itemID.astype(int)
df_unique = df_unique.drop_duplicates('itemID')

len(df_unique)

88497

In [4]:
f_date = lambda x: x[:4] + '-' + x[4:6] + '-' + x[6:]
df_unique['date'] = df_unique.date.astype(str).apply(f_date)

In [5]:
root_path = '/media/intel/m2/evolution/'

In [6]:
df_unique['image_location'] = root_path + df_unique.date + '/' + df_unique.img
f_path = lambda x: '/'.join(x.split('/')[:-2])
df_unique['image_location'] = df_unique.image_location.apply(f_path)

In [18]:
seller_name_list = list(df_unique.seller)

image_location = list(df_unique.image_location)

itemID = list(df_unique.itemID.astype(str))

In [19]:
# import collections
# print [item for item, count in collections.Counter(itemID).items() if count > 1]

In [21]:
target_path = '/media/intel/m2/imgEvo'
try:
    os.mkdir(target_path)
except:
    pass
seller_paths = [os.path.join(target_path, re.sub('[\W_]+', '00', x)) for x in seller_name_list]

In [22]:
def image_seller_path(itemID, image_location, seller_path):
    ori_tar_pair = []
    img_folder_name = [x for x in os.listdir(image_location) if x != 'none']
    img_folder_path = [os.path.join(image_location, x) for x in img_folder_name]
    for i in range(len(img_folder_path)):
        folder_path = img_folder_path[i]
        if os.path.isfile(os.path.join(folder_path, 'large')):
            oriname = 'large'
        elif os.path.isfile(os.path.join(folder_path, 'large.jpg')):
            oriname = 'large.jpg'
        elif os.path.isfile(os.path.join(folder_path, 'tiny')):
            oriname = 'tiny'
        elif os.path.isfile(os.path.join(folder_path, 'tiny.jpg')):
            oriname = 'tiny.jpg'
        else:
            oriname = ''
        if oriname:
            img_ori_path = os.path.join(folder_path, oriname)
            img_tar_path = os.path.join(seller_path,
                                        itemID + 'X' + img_folder_name[i]
                                        + oriname.split('.')[0])
            ori_tar_pair.append([img_ori_path, img_tar_path])
    return ori_tar_pair

In [23]:
time0 = time.time()
uniq_item_count = len(itemID)
img_count = 0
for i in xrange(uniq_item_count):
    if not os.path.isdir(image_location[i]):
        continue
    if not os.path.isdir(seller_paths[i]):
        os.makedirs(seller_paths[i])
    ori_tar_cp_res = image_seller_path(itemID[i], image_location[i],
                                       seller_paths[i])
    for (img_ori_path, img_tar_path) in ori_tar_cp_res:
        copy2(img_ori_path, img_tar_path)
        img_count += 1
print time.time() - time0, img_count

208.974720955 106159


In [1]:
import pandas as pd
import re
import os
import json
import time
from shutil import copy2
from shutil import copyfile
import shutil
from PIL import Image

In [2]:
data = pd.read_json('evolution.json', orient='index', dtype={'itemID':str})

len(data)

904037

In [3]:
data['dayID'] = data.itemID.apply(lambda x: x[8:])

In [4]:
data = data[data.img.notnull()]

In [5]:
data = data.sort_index()

In [6]:
f_date = lambda x: x[:4] + '-' + x[4:6] + '-' + x[6:]
data['date'] = data.date.astype(str).apply(f_date)

In [7]:
root_path = '/media/intel/m2/evolution/'

In [8]:
data['image_location'] = root_path + data.date + '/' + data.img

In [9]:
f_path = lambda x: '/'.join(x.split('/')[:-2])
data['image_location'] = data.image_location.apply(f_path)

In [10]:
data2 = data.copy()

### seller identification

In [33]:
data2 = data2[data2.seller.notnull()]

In [61]:
target_path = '/media/intel/m2/imgEvo'
try:
    os.makedirs(target_path)
except:
    pass

data2['seller_name'] = data2.seller.apply(lambda x: os.path.join(target_path, re.sub('[\W_]+', '00', x)))

valid_item_img = set()
for index, row in data2.iterrows():
    img_f = row.image_location
    if not os.path.isdir(img_f):
        continue
    for img_index in os.listdir(img_f):
        if img_index == 'none':
            continue
        if (row.seller, row.title, row.dayID, img_index) not in valid_item_img:
            if os.path.isfile(os.path.join(img_f, img_index, 'large')):
                oriname = 'large'
            elif os.path.isfile(os.path.join(img_f, img_index, 'large.jpg')):
                oriname = 'large.jpg'
                '''
                elif os.path.isfile(os.path.join(img_f, img_index, 'tiny')):
                    oriname = 'tiny'
                elif os.path.isfile(os.path.join(img_f, img_index, 'tiny.jpg')):
                    oriname = 'tiny.jpg'
                '''
            else:
                continue
            img_path = os.path.join(img_f, img_index, oriname)
            try:
                img = Image.open(img_path)
                img.load()
                img.close()
            except IOError:
                img.close()
                continue        
            if not os.path.isdir(row.seller_name):
                os.makedirs(row.seller_name)
            tar_file = os.path.join(row.seller_name, img_f.split('/')[-1] + 'xxxx' + img_index)
            i = 0
            while os.path.isfile("%sxxxx%d.jpg" % (tar_file, i)):
                i += 1
            copyfile(img_path, "%sxxxx%d.jpg" % (tar_file, i))
            valid_item_img.add((row.seller, row.title, row.dayID, img_index))

In [62]:
target_path = '/media/intel/m2/imgEvodedup'
try:
    os.mkdir(target_path)
except:
    pass

data2['seller_name'] = data2.seller.apply(lambda x: os.path.join(target_path, re.sub('[\W_]+', '00', x)))

for index, row in data2.iterrows():
    img_f = row.image_location
    if not os.path.isdir(img_f):
        continue
    for img_index in os.listdir(img_f):
        if img_index == 'none':
            continue
        if os.path.isfile(os.path.join(img_f, img_index, 'large')):
            oriname = 'large'
        elif os.path.isfile(os.path.join(img_f, img_index, 'large.jpg')):
            oriname = 'large.jpg'
            '''
            elif os.path.isfile(os.path.join(img_f, img_index, 'tiny')):
                oriname = 'tiny'
            elif os.path.isfile(os.path.join(img_f, img_index, 'tiny.jpg')):
                oriname = 'tiny.jpg'
            '''
        else:
            continue
        img_path = os.path.join(img_f, img_index, oriname)
        try:
            img = Image.open(img_path)
            img.load()
            img.close()
        except IOError:
            img.close()
            continue        
        if not os.path.isdir(row.seller_name):
            os.makedirs(row.seller_name)
        tar_file = os.path.join(row.seller_name, img_f.split('/')[-1] + 'xxxx' + img_index)
        i = 0
        while os.path.isfile("%sxxxx%d.jpg" % (tar_file, i)):
            i += 1
        copyfile(img_path, "%sxxxx%d.jpg" % (tar_file, i))

In [63]:
import hashlib
def md5sum(filename, blocksize=65536):
    hash = hashlib.md5()
    with open(filename, "rb") as f:
        for block in iter(lambda: f.read(blocksize), b""):
            hash.update(block)
    return hash.hexdigest()

In [64]:
root_dir = '/media/intel/m2/imgEvodedup'
seller_name = [os.path.join(root_dir, x) for x in os.listdir(root_dir)]
for seller_folder in set(seller_name):
    filenames = map(lambda x: os.path.join(seller_folder, x), os.listdir(seller_folder))
    md5value = {}
    for filename in filenames:
        md5curr = md5sum(filename)
        if md5curr in md5value:
            os.remove(filename)
        else:
            md5value[md5curr] = filename

### category prediction

In [12]:
data3 = data.copy()

In [13]:
data3 = data3[data3.cat.notnull()]

In [14]:
data3.cat.unique()

array([u'Services', u'Fraud Related', u'Electronics', u'Digital Goods',
       u'Guides &amp; Tutorials', u'Drugs', u'Counterfeits',
       u'Miscellaneous', u'Erotica', u'Lab Supplies', u'Weapons',
       u'Jewellery', u'Custom Listings', u'Drug Paraphernalia'], dtype=object)

In [15]:
target_path = '/media/intel/m2/imgEvoCat'
try:
    os.mkdir(target_path)
except:
    pass

data3['cat_name'] = data3.cat.apply(lambda x: os.path.join(target_path, re.sub('[\W_]+', '00', x)))

for index, row in data3.iterrows():
    img_f = row.image_location
    if not os.path.isdir(img_f):
        continue
    for img_index in os.listdir(img_f):
        if img_index == 'none':
            continue
        if os.path.isfile(os.path.join(img_f, img_index, 'large')):
            oriname = 'large'
        elif os.path.isfile(os.path.join(img_f, img_index, 'large.jpg')):
            oriname = 'large.jpg'
            '''
            elif os.path.isfile(os.path.join(img_f, img_index, 'tiny')):
                oriname = 'tiny'
            elif os.path.isfile(os.path.join(img_f, img_index, 'tiny.jpg')):
                oriname = 'tiny.jpg'
            '''
        else:
            continue
        img_path = os.path.join(img_f, img_index, oriname)
        try:
            img = Image.open(img_path)
            img.load()
            img.close()
        except IOError:
            img.close()
            continue        
        if not os.path.isdir(row.cat_name):
            os.makedirs(row.cat_name)
        tar_file = os.path.join(row.cat_name, img_f.split('/')[-1] + 'xxxx' + img_index)
        i = 0
        while os.path.isfile("%sxxxx%d.jpg" % (tar_file, i)):
            i += 1
        copyfile(img_path, "%sxxxx%d.jpg" % (tar_file, i))

In [16]:
import hashlib
def md5sum(filename, blocksize=65536):
    hash = hashlib.md5()
    with open(filename, "rb") as f:
        for block in iter(lambda: f.read(blocksize), b""):
            hash.update(block)
    return hash.hexdigest()

In [17]:
root_dir = '/media/intel/m2/imgEvoCat'
seller_name = [os.path.join(root_dir, x) for x in os.listdir(root_dir)]
for seller_folder in set(seller_name):
    filenames = map(lambda x: os.path.join(seller_folder, x), os.listdir(seller_folder))
    md5value = {}
    for filename in filenames:
        md5curr = md5sum(filename)
        if md5curr in md5value:
            os.remove(filename)
        else:
            md5value[md5curr] = filename